In [ ]:
#general packages
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from collections import Counter
#plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
#custom function
from false_positive_analysis import percent_false_positive
%config InlineBackend.figure_format='retina'

In [ ]:
#read in final mtx
#grab all channels
all_channels = []

for c in [1]:
    mtx = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/150genes3bind_040622/notebook_pyfiles/genebycell/final_thresh7_11p52_33_heg_filt/final/genebycell_{c}.csv", index_col=0)
    mtx = mtx.T
    mtx = mtx[mtx.sum(axis=1)>100].T
    mtx = mtx[mtx.sum(axis=1)>1]
    all_channels.append(mtx)

mtx = pd.concat(all_channels, axis=1).fillna(0)

In [ ]:
#take a look
mtx

In [ ]:
#read in rnaseq data
rnaseq = pd.read_csv("./RNAseq_files/nih3t3_FPKM_CCS.csv")
rnaseq.columns = ["Genes","FPKM"]

In [ ]:
#codebook
codebook = pd.read_csv("/groups/CaiLab/personal/Lex/raw/150genes3bind_040622/barcode_key/convert_barcode_key/codebook_string_561.csv", index_col=0)
#separate into true and false codebook
fakebook = codebook[codebook.index.str.startswith("fake")]
codebook = codebook.drop(fakebook.index)

In [ ]:
#calculate fdr
fp, fake, norm_fpr= percent_false_positive(mtx, codebook, fakebook)
percent_fp = fp["FP raw"].mean()
mean_counts = fp["total_counts"].mean()
sum_counts = fp["total_counts"].sum()
fp_list = [percent_fp,norm_fpr,mean_counts,sum_counts]

In [ ]:
#take a look at fdr results
df_stats = pd.DataFrame(fp_list).T
df_stats.columns = ["percent fp","false positive rate","mean counts", "total sum"]
df_stats

In [ ]:
#convert data to pseudobulk rnaseq data
bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
bulk.columns = ["Genes", "Counts"]
bulk["Genes"] = bulk["Genes"].str.lower()
rnaseq["Genes"] = rnaseq["Genes"].str.lower()
#merge
comb_1 = pd.merge(rnaseq,bulk)
#pearson's correlation
r = pearsonr(comb_1["FPKM"],comb_1["Counts"])
r = round(r[0],2)

In [ ]:
#get log2 + 1
comb_1["Log Counts"] = np.log2(comb_1["Counts"]+1)
comb_1["Log FPKM"] = np.log2(comb_1["FPKM"]+1)

In [ ]:
#plot rnaseq correlations
plt.plot(comb_1["Log Counts"], comb_1["Log FPKM"], 'bo', alpha=0.5)
plt.title("Channel 647 nm", fontweight="bold")
plt.ylabel("Bulk RNAseq Log2(FPKM+1)", fontsize=12)
plt.xlabel("Pseudobulk Log2(Counts+1)", fontsize=12)
plt.annotate(f"Pearson's r= {round(r,2)}", (0.5,10), fontsize=12)
sns.despine()
plt.show()

In [ ]:
#read in smfish 
smfish = pd.read_csv("./smFISH_results.csv")

In [ ]:
#convert to pseudobulk results 
smfish_df = pd.DataFrame(smfish.T.mean(axis=1))
smfish_df = smfish_df.reset_index()
smfish_df.columns = ["Genes", "smFISH Counts"]
smfish_df["Genes"] = smfish_df["Genes"].str.lower()

In [ ]:
#convert data to pseudobulk rnaseq data
bulk = pd.DataFrame(mtx.mean(axis=1)).reset_index()
bulk.columns = ["Genes", "Counts"]
bulk["Genes"] = bulk["Genes"].str.lower()
#merge
comb_2 = pd.merge(smfish_df,bulk)
#calculate info
x = comb_2["smFISH Counts"].values
x_t = np.vstack([x, np.zeros(len(x))]).T
y = comb_2["Counts"].values
m,c = np.linalg.lstsq(x_t, y, rcond=None)[0]
r = pearsonr(x,y)[0]

In [ ]:
#show smfish correlation
plt.plot(x, y, 'bo', alpha=0.5)
plt.plot(x, x*m, c = "k")
plt.title("Channel 647 nm", fontweight="bold")
plt.xlabel("Average smFISH counts")
plt.ylabel("Average LANTERN Counts")
plt.annotate(f"Pearson's r= {round(r,2)}", (0,230), fontsize=12)
plt.annotate(f"Efficiency = {round(m,2)}", (0,210), fontsize=12)
sns.despine()
plt.show()

In [ ]:
#plot correlation and efficiency
plt.scatter(x = np.log2(comb_2["smFISH Counts"]), y = np.log2(comb_2["Counts"]), c="b", alpha=0.5)
plt.xlabel("Log2(Average smFISH Counts)", fontsize=12)
plt.ylabel("Log2(Average LANTERN Counts)", fontsize=12)
plt.annotate(f"Pearson's r= {round(r,2)}", (-1,8), fontsize=12)
plt.annotate(f"Efficiency = {round(m,2)}", (-1,7.3), fontsize=12)
plt.title("Channel 647 nm", fontweight="bold")
sns.despine()

# Off target properties for potential filtering

In [ ]:
#read in one of the gene mapped locations file
locations = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/150genes3bind_040622/notebook_pyfiles/decoded/final_thresh8_11p52_33_heg/Channel_1/genes_in_cells/Pos_0/gene_locations.csv", index_col=0)

In [ ]:
#separate trues and fakes
fake = locations[locations["genes"].str.startswith("fake")]
trues = locations.drop(fake.index)

In [ ]:
#plot histograms of ambiguity score
_max=max(np.max(fake["ambiguity score"]),np.max(trues["ambiguity score"]))
_min=min(np.min(fake["ambiguity score"]),np.min(trues["ambiguity score"]))

plt.hist(trues["ambiguity score"], bins=20, alpha= 0.5, range = (_min,_max), label = "On")
plt.hist(fake["ambiguity score"], bins=20, color= "red", alpha=0.5, range = (_min,_max), label = "Off")
plt.xlim(0,10)
plt.xlabel("ambiguity score")
plt.ylabel("Counts")
sns.despine()
plt.legend()
plt.show()

In [ ]:
#plot histograms of brightness
_max=max(np.max(fake["brightness"]),np.max(trues["brightness"]))
_min=min(np.min(fake["brightness"]),np.min(trues["brightness"]))

plt.hist(trues["brightness"], bins=50, alpha= 0.5, range = (_min,_max), label = "On")
plt.hist(fake["brightness"], bins=50, color= "red", alpha=0.5, range = (_min,_max), label = "Off")
#plt.xlim(0,10)
plt.xlabel("Flux")
plt.ylabel("Counts")
sns.despine()
plt.legend()
plt.show()

In [ ]:
#plot histogram of sizes
_max=max(np.max(fake["size"]),np.max(trues["size"]))
_min=min(np.min(fake["size"]),np.min(trues["size"]))

plt.hist(trues["size"], bins=30, alpha= 0.5, range = (_min,_max), label = "On")
plt.hist(fake["size"], bins=30, color= "red", alpha=0.5, range = (_min,_max), label = "Off")
#plt.xlim(0,10)
plt.xlabel("Size")
plt.ylabel("Counts")
sns.despine()
plt.legend()
plt.show()

In [ ]:
#plot size and brightness
plt.scatter(trues["brightness"],trues["size"], label = "On",c="blue", alpha=0.5)
plt.scatter(fake["brightness"],fake["size"], label = "Off",c="red", alpha=0.5)
#plt.xlim(0,10)
plt.xlabel("Flux")
plt.ylabel("Size")
sns.despine()
plt.legend()
plt.show()

In [ ]:
#filter
plt.scatter(trues[trues["brightness"]>2.5]["brightness"],trues[trues["brightness"]>2.5]["size"], label = "On",c="blue", alpha=0.5)
plt.scatter(fake[fake["brightness"]>2.5]["brightness"],fake[fake["brightness"]>2.5]["size"], label = "Off",c="red", alpha=0.5)
plt.scatter(trues[trues["brightness"]<2.5]["brightness"],trues[trues["brightness"]<2.5]["size"],c="gray",  alpha=0.5)
plt.scatter(fake[fake["brightness"]<2.5]["brightness"],fake[fake["brightness"]<2.5]["size"],c="gray", alpha=0.5)
#plt.xlim(0,10)
plt.xlabel("Flux")
plt.ylabel("Size")
plt.axvline(2.5, c = "black")
sns.despine()
plt.legend()
plt.show()

In [ ]:
#write out filtered gene locations
from pathlib import Path
ch=1
pos_list = np.arange(0,40,1)
for pos in pos_list:
    path = Path(f"/groups/CaiLab/personal/Lex/raw/150genes3bind_040622/notebook_pyfiles/decoded/final_thresh7_11p52_33_heg/Channel_1/genes_in_cells/Pos_{pos}/gene_locations.csv")
    locations = pd.read_csv(str(path), index_col=0)
    locations=locations[locations["brightness"]>2.5]
    output_path = path.parent /"gene_locations_filt.csv"
    locations.to_csv(str(output_path))

# Identify problematic hybs

In [ ]:
#collapse into gene counts
counts_df = Counter(fake["genes"])
#change to df
counts_df = pd.DataFrame.from_dict(counts_df, orient='index').reset_index()
counts_df

In [ ]:
#check codebook
codebook = pd.read_csv("/groups/CaiLab/personal/Lex/raw/150genes3bind_040622/barcode_key/codebook_647nm.csv", index_col=0)

In [ ]:
#lowercase gene names
codebook.index = codebook.index.str.lower()

In [ ]:
#get top 30
maj_fakes = counts_df.sort_values(0, ascending=False).head(30)

In [ ]:
#get troubled hybs
troubled_hybs = []
for fakes in maj_fakes["index"]:
    troubled_hybs.append(codebook.loc[fakes].values.tolist())

In [ ]:
#convert to array
hybstocheck = np.array(troubled_hybs)

In [ ]:
#take a look
hybstocheck

In [ ]:
#get mode for each round
bad_hybs = []
for i in range(len(hybstocheck[0])):
    vals,counts = np.unique(hybstocheck[:,i], return_counts=True)
    index = np.argmax(counts)
    bad_hyb = vals[index]
    bad_hybs.append(bad_hyb)

print(f"Following hybs by rounds are problematic:{bad_hybs}" )